In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_date_me_directory():
    # Fetch the webpage
    url = 'https://dateme.directory/browse'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the webpage: {e}")
        return None

    # Parse HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'id': 'docs'})
    
    if not table:
        print("Could not find the data table")
        return None

    # Prepare data storage
    data = []
    columns = ['Name', 'Age', 'Gender', 'InterestedIn', 'Style', 'Location', 
               'LocationFlexibility', 'Contact', 'LastUpdated']

    # Extract table rows
    for row in table.find('tbody').find_all('tr'):
        cells = row.find_all('td')
        if len(cells) != 9:
            continue

        # Extract data from each cell
        row_data = {
            'Name': cells[0].find('a').text.strip() if cells[0].find('a') else cells[0].div.text.strip(),
            'Age': cells[1].div.text.strip(),
            'Gender': cells[2].div.text.strip(),
            'InterestedIn': ', '.join([span.text.strip() for span in cells[3].div.find_all('span')]) or cells[3].div.text.strip(),
            'Style': ', '.join([span.text.strip() for span in cells[4].div.find_all('span')]) or cells[4].div.text.strip(),
            'Location': ', '.join([span.text.strip() for span in cells[5].div.find_all('span')]) or cells[5].div.text.strip(),
            'LocationFlexibility': cells[6].div.text.strip(),
            'Contact': cells[7].div.text.strip(),
            'LastUpdated': cells[8].div.text.strip()
        }
        data.append(row_data)

    # Create DataFrame
    df = pd.DataFrame(data, columns=columns)
    return df

if __name__ == "__main__":
    df = scrape_date_me_directory()
    if df is not None:
        print("Successfully scraped data:")
        print(df.head())
        # To save to CSV:
        # df.to_csv('date_me_directory.csv', index=False)
    else:
        print("Failed to scrape data")